<a href="https://colab.research.google.com/github/codesigningwiththemachine/dvww2022/blob/main/VideoClassifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Setup**

In [ ]:
#@title ## Install and load requirements
#@markdown ---

#@markdown Install / update Fastai and load other requirements

! [ -e /content ] && pip install -Uqq fastai  # upgrade fastai on colab
import pandas as pd
from fastai.vision.all import *
from fastai.collab import *
from fastai.basics import *


In [ ]:
#@title ## Mount Google Drive
#@markdown ---
#@markdown Mount google drive and load project folder
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
#@title ## Set Project folder
#@markdown ---
#@markdown Go to project folder

Project_Folder = '/content/drive/MyDrive/OWN/CHIBAELISAVA/ImageClas' #@param {type:"string"}
%cd $Project_Folder

from pathlib import Path
path = Path(Project_Folder)

# **Single-label classification**

In [ ]:
#@title ## Load labels
Labels = "/content/drive/MyDrive/OWN/CHIBAELISAVA/ImageClas/Imagelabels.csv" #@param {type:"string"}

#Pandas dataframe from csv file
df = pd.read_csv(Labels)
df.head()

In [ ]:
#@title ## Convert Dataset
#@markdown ---
#@markdown Convert data into a dataloader object to be used by fastai.
#@markdown Data is prepared for our image classification model and organized in batches. 

dls = ImageDataLoaders.from_df(df, path, folder='.',fn_col=0,item_tfms=Resize(256,ResizeMethod.Squish))
dls.show_batch()

In [ ]:
#@title ## Create and train an image classification model
#@markdown ---

#@markdown Using cnn_learner to prepare and train a language model based on our data. A pretrained resnet34 is used as a base.
Epochs =  1#@param {type:"number"}

#learn = cnn_learner(dls, resnet34, metrics=accuracy)
#learn.lr_find()
learn.fit_one_cycle(Epochs, 2e-02)

In [ ]:
#@title ## Upload Images and get prediction
#@markdown ---

#@markdown Select a set of images (.jpg, .png). And get a prediction from our model 

from google.colab import files
uploaded = files.upload()
for image in list(uploaded.keys()):
  print(learn.predict(image)[0])

In [ ]:
#@title ## Upload video
#@markdown ---

#@markdown Select a set of images (.jpg, .png). And get a prediction from our model 
import cv2

imagepath = "/content/drive/MyDrive/OWN/CHIBAELISAVA/ImageClas/videos" #@param {type: "string"}

from google.colab import files
uploaded = files.upload()
for video in list(uploaded.keys()):
  vidcap = cv2.VideoCapture(video)
  success,image = vidcap.read()
  count = 0
  while success:
    cv2.imwrite(imagepath+"/frame%d.jpg" % count, image)     # save frame as JPEG file      
    success,image = vidcap.read()
    print('Read a new frame: ', success)
    count += 1

import time
time.sleep(20) 

from os import walk
filenames = next(walk(imagepath), (None, None, []))[2]  # [] if no file

for image in filenames:
  print(learn.predict(imagepath +"/" +image)[0])


Saving giphy.mp4 to giphy (2).mp4


# **Export and Load our model**

In [ ]:
#@title ## Export our model
#@markdown ---

Model = "test.pkl" #@param {type:"string"}
learn.export(Model)

In [ ]:
#@title ## Load our pre trained model
#@markdown ---

Model = "test.pkl" #@param {type:"string"}
learn = load_learner(Model)

# **Deploy Model**

In [ ]:
%%capture
#@title ## Install and load requirements
#@markdown ---

#@markdown Install and setup requirements to serve model online

! [ -e /content ] && pip install -Uqq fastai  # upgrade fastai on colab
!pip install flask
!pip install flask-ngrok
!pip install flask-bootstrap
!pip install jsonpickle
!pip install -U flask-cors

#!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
#!unzip ngrok-stable-linux-amd64.zip
#!./ngrok authtoken 22aZcoUzJr4EMX5EXzWI1kbzawH_55G5fBvzCn8mtyhW9Woai



In [ ]:
#@title ## Run web server 
#@markdown ---

#@markdown 

from flask_ngrok import run_with_ngrok
from flask import Flask, request, Response
from flask_cors import CORS

import jsonpickle
import numpy as np
import cv2
import base64
from io import BytesIO
from PIL import Image



# Initialize the Flask application
app = Flask(__name__)
CORS(app)

# route http posts to this method
run_with_ngrok(app)
@app.route('/api/predict', methods=['POST'])
def test():
    content = request.json
    file = content['image']
    starter = file.find(',')
    image_data = file[starter+1:]
    image_data = bytes(image_data, encoding="ascii")
    im = Image.open(BytesIO(base64.b64decode(image_data)))
    im.save('image.png')
    prediction = learn.predict('image.png')[0]
    print(prediction)
    #print(prediction)
    response = {'Prediction': '{}'.format(prediction)}
    response_pickled = jsonpickle.encode(response)
    return Response(response=response_pickled, status=200, mimetype="application/json")



if __name__ == '__main__':
   app.run()

In [ ]:
#@title Our url

path = "https://7f83-35-193-186-113.ngrok.io/api/predict" #@param {type: 'string'}

# **URL to test: [Prediction](https://editor.p5js.org/codesigningwiththemachine/sketches/pIbxAXGtA)**